In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from post_clustering import spectral_clustering, acc, nmi
import scipy.io as sio
import math

# Conv2dSamePad, ConvTranspose2dSamePad, ConvAE, SelfExpression classes remain unchanged

class Conv2dSamePad(nn.Module):
    def __init__(self, kernel_size, stride):
        super(Conv2dSamePad, self).__init__()
        self.kernel_size = kernel_size if isinstance(kernel_size, (list, tuple)) else [kernel_size, kernel_size]
        self.stride = stride if isinstance(stride, (list, tuple)) else [stride, stride]

    def forward(self, x):
        in_height = x.size(2)
        in_width = x.size(3)
        out_height = math.ceil(float(in_height) / float(self.stride[0]))
        out_width = math.ceil(float(in_width) / float(self.stride[1]))
        pad_along_height = max((out_height - 1) * self.stride[0] + self.kernel_size[0] - in_height, 0)
        pad_along_width = max((out_width - 1) * self.stride[1] + self.kernel_size[1] - in_width, 0)
        pad_top = pad_along_height // 2
        pad_left = pad_along_width // 2
        pad_bottom = pad_along_height - pad_top
        pad_right = pad_along_width - pad_left

        # Apply padding
        return F.pad(x, [pad_left, pad_right, pad_top, pad_bottom], mode='constant', value=0)


class ConvTranspose2dSamePad(nn.Module):
    def __init__(self, kernel_size, stride):
        super(ConvTranspose2dSamePad, self).__init__()
        self.kernel_size = kernel_size if type(kernel_size) in [list, tuple] else [kernel_size, kernel_size]
        self.stride = stride if type(stride) in [list, tuple] else [stride, stride]

    def forward(self, x):
        in_height = x.size(2)
        in_width = x.size(3)
        pad_height = self.kernel_size[0] - self.stride[0]
        pad_width = self.kernel_size[1] - self.stride[1]
        pad_top = pad_height // 2
        pad_bottom = pad_height - pad_top
        pad_left = pad_width // 2
        pad_right = pad_width - pad_left
        return x[:, :, pad_top:in_height - pad_bottom, pad_left: in_width - pad_right]



class SelfExpression(nn.Module):
    def __init__(self, n):
        super(SelfExpression, self).__init__()
        self.Coefficient = nn.Parameter(1.0e-8 * torch.ones(n, n, dtype=torch.float32), requires_grad=True)

    def forward(self, x):
        y = torch.matmul(self.Coefficient, x)
        return y
class ConvAE(nn.Module):
    def __init__(self, channels, kernels):
        """
        :param channels: a list containing all channels including the input image channel (1 for gray, 3 for RGB)
        :param kernels:  a list containing all kernel sizes, it should satisfy: len(kernels) = len(channels) - 1.
        """
        super(ConvAE, self).__init__()
        assert isinstance(channels, list) and isinstance(kernels, list)
        self.encoder = nn.Sequential()
        for i in range(1, len(channels)):
            #  Each layer will divide the size of feature map by 2
            self.encoder.add_module('pad%d' % i, Conv2dSamePad(kernels[i - 1], 2))
            self.encoder.add_module('conv%d' % i,
                                    nn.Conv2d(channels[i - 1], channels[i], kernel_size=kernels[i - 1], stride=2))
            self.encoder.add_module('relu%d' % i, nn.ReLU(True))

        self.decoder = nn.Sequential()
        channels = list(reversed(channels))
        kernels = list(reversed(kernels))
        for i in range(len(channels) - 1):
            # Each layer will double the size of feature map
            self.decoder.add_module('deconv%d' % (i + 1),
                                    nn.ConvTranspose2d(channels[i], channels[i + 1], kernel_size=kernels[i], stride=2))
            self.decoder.add_module('padd%d' % i, ConvTranspose2dSamePad(kernels[i], 2))
            self.decoder.add_module('relud%d' % i, nn.ReLU(True))

    def forward(self, x):
        h = self.encoder(x)
        y = self.decoder(h)
        return y
class DSCNet(nn.Module):
    def __init__(self, channels, kernels, num_sample):
        super(DSCNet, self).__init__()
        self.n = num_sample
        self.ae = ConvAE(channels, kernels)
        self.self_expression = SelfExpression(self.n)

    def forward(self, x):
        z = self.ae.encoder(x)
        shape = z.shape
        z = z.view(self.n, -1)
        z_recon = self.self_expression(z)
        z_recon_reshape = z_recon.view(shape)
        x_recon = self.ae.decoder(z_recon_reshape)
        z_recon_reshape = z_recon.view(shape)
        

# 在这里手动调整张量的大小
        #x_recon = x_recon[:, :, :145, :145]  # 假设你需要裁剪到 (145, 145)

        return x_recon, z, z_recon

    def loss_fn(self, x, x_recon, z, z_recon, weight_coef, weight_selfExp):
        loss_ae = F.mse_loss(x_recon, x, reduction='sum')
        loss_coef = torch.sum(torch.pow(self.self_expression.Coefficient, 2))
        loss_selfExp = F.mse_loss(z_recon, z, reduction='sum')
        loss = loss_ae + weight_coef * loss_coef + weight_selfExp * loss_selfExp
        return loss

def train(model, x, y, epochs, lr=1e-3, weight_coef=1.0, weight_selfExp=150, device='cuda',
          alpha=0.04, dim_subspace=12, ro=8, show=10):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    if not isinstance(x, torch.Tensor):
        x = torch.tensor(x, dtype=torch.float32, device=device)
    x = x.to(device)
    if isinstance(y, torch.Tensor):
        y = y.to('cpu').numpy()
    K = len(np.unique(y))
    for epoch in range(epochs):
        x_recon, z, z_recon = model(x)
        loss = model.loss_fn(x, x_recon, z, z_recon, weight_coef=weight_coef, weight_selfExp=weight_selfExp)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % show == 0 or epoch == epochs - 1:
            C = model.self_expression.Coefficient.detach().to('cpu').numpy()
            y_pred = spectral_clustering(C, K, dim_subspace, alpha, ro)
            print('Epoch %02d: loss=%.4f, acc=%.4f, nmi=%.4f' %
                  (epoch, loss.item() / y_pred.shape[0], acc(y, y_pred), nmi(y, y_pred)))

def run_on_indian_pines():
    # Load Indian Pines dataset
    
    # 加载Indian Pines数据集
    data_mat = sio.loadmat('indianpines_dataset.mat')
    labels_mat = sio.loadmat('indianpines_gt.mat')

    data = data_mat['pixels']
    labels = labels_mat['pixels']

    print("labels的初始维度：",labels.shape)
    # 交换标签的第一维度和第二维度
    labels = np.transpose(labels, (1, 0))
    x, y = data.reshape((-1, 1, 145, 145)), labels
    y = np.squeeze(y)  # y in [0, 1, ..., K-1]
    # Network parameters

    num_sample = x.shape[0]
    channels = [1, 200]
    kernels = [5]
    epochs = 40
    weight_coef = 1.0
    weight_selfExp = 75
    alpha = 0.04
    dim_subspace = 12
    ro = 8

    # Initialize model and move to device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = DSCNet(channels=channels, kernels=kernels, num_sample=num_sample)
    model.to(device)

    # Train the model
    train(model, x, y, epochs, weight_coef=weight_coef, weight_selfExp=weight_selfExp,
          alpha=alpha, dim_subspace=dim_subspace, ro=ro, device=device)

    # Save the model
    torch.save(model.state_dict(), 'dscnet_indian_pines.ckpt')


run_on_indian_pines()


labels的初始维度： (1, 21025)


TypeError: ConvTranspose2dSamePad.__init__() missing 2 required positional arguments: 'kernel_size' and 'stride'

In [15]:

for i in range(1000,99,-1):
    if(555555%i==0):
       print(i)
       break



777
